## Predicting Life Satisfaction
### Exploratory Data Analysis  
  
#### Objectives:  
* Analysis of missing observations 
* Analysis of categorical/numeric features 
* 1 way/ 2 way variable plots (histograms, scatterplots, etc.) 
* Correlation analysis

### From Kaggle:
#### File descriptions  
* train.csv - the training set with some preprocesssing of values.  
* test.csv - the test set with some preprocesssing of values.  
* train_raw.csv - the training set with original responses (no preprocessing).  
* test_raw.csv - the test set with original responses (no preprocessing).  
* codebook_compact.txt - short descriptions of each data field.  
* codebook_long.txt - long descriptions of each field.  
* sample_submission.csv - a sample submission file in the correct format. Change values in the Predicted column to estimates of the probability the person is very satisfied and submit to receive a leaderboard score.  
  
#### Data fields
* sample_submission.csv - survey respondent id
* v1-v270 - survey response fields
* cntry - survey respondent country
* satisfied - whether (1) or not (0) the survey respondent is 'very satisfied' with their life (training set only)

In [25]:
# Import packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Set options
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [ ]:
# Import data
train_raw = pd.read_csv("../01-data/train_raw.csv", low_memory = False)
train = pd.read_csv("../01-data/train.csv", low_memory = False)
test_raw = pd.read_csv("../01-data/test_raw.csv", low_memory = False)
test = pd.read_csv("../01-data/test.csv", low_memory = False)